## Imports & functions:

In [93]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime

newIDs = [24, 25, 5, 6, 27, 23, 26, 7, 11, 13, 14, 15, 16, 17, 18, 19, 21, 22, 8, 9, 10, 1, 3, 2, 4]

def cleanup(data: str):
    start = data.find("<pre>") + 5
    end = data.find("</pre>")
    data = data[start:end]
    data = data.replace(" ", "")
    data = data.replace("\\n", "\n")
    return data


def downloadData(ID):
    for id in ID:
        url = f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={newIDs[id-1]}&year1=1981&year2=2020&type=Mean"
        filename = f"{id}"
        with requests.get(url, headers={"Accept": "text/plain"}) as plain:
            data = str(plain.content)
            data = cleanup(data)
        with open(f"Data/{filename}.csv", "w") as output:
            output.write(f"Year,Week,1,2,3,4,VHI\n")
            output.write(data)
        
def createFrame(ID):
    tempframes = []
    for id in ID:
        temp = pd.read_csv(f"Data/{id}.csv", header=0, index_col=False)
        temp["Province"] = id
        tempframes.append(temp)
    df = pd.concat(tempframes)
    return df

## Creating "Global" frame:

In [94]:
downloadData(range(1, 4))
global_df = createFrame(range(1, 4))

global_df

,Year,Week,1,2,3,4,VHI,Province
0,1982,1,0.068,263.59,63.47,28.34,45.90,1
1,1982,2,0.074,265.78,67.62,23.05,45.34,1
2,1982,3,0.076,267.19,69.37,20.40,44.88,1
3,1982,4,0.075,268.57,65.26,17.93,41.60,1
4,1982,5,0.072,269.24,58.58,20.00,39.29,1
...,...,...,...,...,...,...,...,...
2023,2020,48,-1.000,-1.00,-1.00,-1.00,-1.00,3
2024,2020,49,-1.000,-1.00,-1.00,-1.00,-1.00,3
2025,2020,50,-1.000,-1.00,-1.00,-1.00,-1.00,3
2026,2020,51,-1.000,-1.00,-1.00,-1.00,-1.00,3


In [95]:
global_df = (globalframe.where(globalframe["VHI"] != -1)
                        .dropna())

global_df.tail()

,Year,Week,1,2,3,4,VHI,Province
1987,2020.0,12.0,0.250,281.89,86.80,23.05,54.92,2.0
1988,2020.0,13.0,0.261,283.95,84.96,22.11,53.53,2.0
1989,2020.0,14.0,0.269,285.99,79.97,19.49,49.73,2.0
1990,2020.0,15.0,0.279,287.32,73.37,20.30,46.84,2.0
1991,2020.0,16.0,0.284,287.98,62.75,30.58,46.66,2.0


## Creating table for country:

In [74]:
country_df = (global_df.groupby("Year")
                        .agg({"VHI": ["min", "max", "mean"]}))
country_df.head()

VHI                  
          min    max       mean
Year                           
1982.0  19.92  59.42  39.092404
1983.0  28.87  54.04  40.204712
1984.0  22.11  76.06  46.448980
1985.0  27.27  60.35  42.556591
1986.0  23.15  53.25  37.140673

## Creating table for provinces:

In [69]:
province_df = (global_df.groupby(["Year", "Province"])
                        .agg({"VHI": ["min", "max", "mean"]}))
province_df.head()

VHI                  
                   min    max       mean
Year   Province                         
1982.0 1.0       19.92  59.42  40.285962
       2.0       23.55  53.88  37.898846
1983.0 1.0       31.16  54.04  41.273846
       2.0       28.87  52.61  39.135577
1984.0 1.0       31.54  76.06  53.313061

## Creating table with month column:

In [70]:
vhimonth_df = global_df
vhimonth_df["Month"] = 7 * vhimonth_df["Week"] // 31 + 1
vhimonth_df = (vhimonth_df.groupby("Month")
                            .agg({"VHI": ["min", "max"]}))
vhimonth_df.head()

VHI       
         min    max
Month              
1.0    22.58  70.28
2.0    25.03  67.28
3.0    27.08  61.58
4.0    27.02  75.51
5.0    28.84  81.44